# Connect to MLflow in Databricks

In [ ]:
import mlflow
import mlflow.tracking._model_registry.utils
from dotenv import load_dotenv

load_dotenv()

# The login and mlflow model registery are set this way because of local development. If you are running this in Databricks, you can remove the login and use the regualr mlflow registry.

mlflow.login()

mlflow.tracking._model_registry.utils._get_registry_uri_from_spark_session = (
    lambda: "databricks-uc"
)

mlflow.set_experiment("/PyFuncReranker")

In [ ]:
model_inputs = {
    "query": "Organic skincare products for sensitive skin",
    "documents": [
        "Organic skincare for sensitive skin with aloe vera and chamomile.",
        "New makeup trends focus on bold colors and innovative techniques",
        "Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille",
        "Neue Make-up-Trends setzen auf kräftige Farben und innovative Techniken",
        "Cuidado de la piel orgánico para piel sensible con aloe vera y manzanilla",
        "Las nuevas tendencias de maquillaje se centran en colores vivos y técnicas innovadoras",
        "针对敏感肌专门设计的天然有机护肤产品",
        "新的化妆趋势注重鲜艳的颜色和创新的技巧",
        "敏感肌のために特別に設計された天然有機スキンケア製品",
        "新しいメイクのトレンドは鮮やかな色と革新的な技術に焦点を当てています",
    ],
}

model_outputs = [
    {
        "corpus_id": 6,
        "score": 0.9258,
        "text": "针对敏感肌专门设计的天然有机护肤产品",
    },
    {
        "corpus_id": 8,
        "score": 0.8438,
        "text": "敏感肌のために特別に設計された天然有機スキンケア製品",
    },
    {
        "corpus_id": 0,
        "score": 0.8281,
        "text": "Organic skincare for sensitive skin with aloe vera and chamomile.",
    },
    {
        "corpus_id": 4,
        "score": 0.7617,
        "text": "Cuidado de la piel orgánico para piel sensible con aloe vera y manzanilla",
    },
    {
        "corpus_id": 2,
        "score": 0.6289,
        "text": "Bio-Hautpflege für empfindliche Haut mit Aloe Vera und Kamille",
    },
    {
        "corpus_id": 9,
        "score": 0.1128,
        "text": "新しいメイクのトレンドは鮮やかな色と革新的な技術に焦点を当てています",
    },
    {
        "corpus_id": 5,
        "score": 0.0996,
        "text": "Las nuevas tendencias de maquillaje se centran en colores vivos y técnicas innovadoras",
    },
    {
        "corpus_id": 1,
        "score": 0.0952,
        "text": "New makeup trends focus on bold colors and innovative techniques",
    },
    {
        "corpus_id": 3,
        "score": 0.0825,
        "text": "Neue Make-up-Trends setzen auf kräftige Farben und innovative Techniken",
    },
    {
        "corpus_id": 7,
        "score": 0.0593,
        "text": "新的化妆趋势注重鲜艳的颜色和创新的技巧",
    },
]


signature = mlflow.models.infer_signature(
    model_input=model_inputs,
    model_output=model_outputs,
)

signature

# Register model

In [ ]:
# https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html?highlight=pythonmodel#mlflow.pyfunc.PythonModel

with mlflow.start_run():
    model_info = mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model="PyFuncReranker.py",
        pip_requirements="../requirements.txt",
        input_example=model_inputs,
        signature=signature,
    )

loaded = mlflow.pyfunc.load_model(model_info.model_uri)

# Test model

In [ ]:
result = loaded.predict(model_inputs)
result